In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn import preprocessing
min_max_scaler = lambda x: (x-np.min(x))/(np.max(x)-np.min(x))

In [4]:
 pollution_file = 'pollution_2000_2021.csv'

In [5]:
v = pd.read_csv(pollution_file,
               sep=',')
v

,Date,Year,Month,Day,Address,State,County,City,O3 Mean,O3 1st Max Value,...,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI
0,2000-01-01,2000,1,1,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.019765,0.040,...,23,25.0,3.000000,9.0,21,13.0,19.041667,49.0,19,46
1,2000-01-02,2000,1,2,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.015882,0.032,...,0,26.0,1.958333,3.0,22,4.0,22.958333,36.0,19,34
2,2000-01-03,2000,1,3,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.009353,0.016,...,8,28.0,5.250000,11.0,19,16.0,38.125000,51.0,8,48
3,2000-01-04,2000,1,4,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.015882,0.033,...,23,34.0,7.083333,16.0,8,23.0,40.260870,74.0,8,72
4,2000-01-05,2000,1,5,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,0.007353,0.012,...,2,42.0,8.708333,15.0,7,21.0,48.450000,61.0,22,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608694,2021-06-26,2021,6,26,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,0.040588,0.049,...,0,1.0,0.409091,0.7,2,0.0,1.281818,2.1,0,2
608695,2021-06-27,2021,6,27,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,0.037882,0.042,...,0,1.0,0.325000,0.4,4,0.0,1.104167,2.3,20,2
608696,2021-06-28,2021,6,28,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,0.039765,0.050,...,0,1.0,0.326087,0.5,2,0.0,1.678261,5.8,23,5
608697,2021-06-29,2021,6,29,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,0.049176,0.058,...,0,1.0,0.317391,0.5,20,0.0,2.934783,7.4,0,7


In [6]:
v.dtypes

Date                  object
Year                   int64
Month                  int64
Day                    int64
Address               object
State                 object
County                object
City                  object
O3 Mean              float64
O3 1st Max Value     float64
O3 1st Max Hour        int64
O3 AQI                 int64
CO Mean              float64
CO 1st Max Value     float64
CO 1st Max Hour        int64
CO AQI               float64
SO2 Mean             float64
SO2 1st Max Value    float64
SO2 1st Max Hour       int64
SO2 AQI              float64
NO2 Mean             float64
NO2 1st Max Value    float64
NO2 1st Max Hour       int64
NO2 AQI                int64
dtype: object

In [7]:
la = v[v['City'] == 'Los Angeles']

In [8]:
la = la.copy()

In [9]:
la['Date'] = pd.to_datetime(la['Date']) # transform 'date' to datetime, helping for group.

In [10]:
la1 = la.groupby(['Date']).mean() 

In [11]:
la1

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI
Date,,,,,,,,,,,,,,,,,,,
2000-03-01,2000.0,3.0,1.0,0.011824,0.0270,9.0,25.0,1.004167,1.60,8.0,18.0,0.000000,0.00,0.0,0.0,33.521739,48.00,22.0,45.0
2000-03-02,2000.0,3.0,2.0,0.011235,0.0250,10.0,23.0,1.537500,2.20,4.0,25.0,0.173913,1.00,19.0,1.0,44.173913,55.00,18.0,52.0
2000-03-03,2000.0,3.0,3.0,0.015294,0.0250,10.0,23.0,0.729167,1.00,0.0,11.0,0.391304,3.00,5.0,4.0,32.608696,42.00,18.0,40.0
2000-03-04,2000.0,3.0,4.0,0.021647,0.0290,12.0,27.0,0.712500,1.00,8.0,11.0,0.260870,3.00,5.0,4.0,31.347826,42.00,10.0,40.0
2000-03-05,2000.0,3.0,5.0,0.018882,0.0320,11.0,30.0,0.404167,0.80,23.0,9.0,0.086957,2.00,5.0,3.0,16.956522,42.00,22.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-26,2021.0,6.0,26.0,0.028435,0.0448,9.8,41.2,0.240000,0.30,4.0,3.4,0.553333,0.98,7.2,0.8,11.685834,21.62,18.4,20.4
2021-06-27,2021.0,6.0,27.0,0.034377,0.0532,9.8,55.0,0.263333,0.34,0.8,4.2,0.544167,0.80,4.8,0.8,10.852500,21.80,5.4,20.4
2021-06-28,2021.0,6.0,28.0,0.036717,0.0552,10.2,58.6,0.259167,0.36,7.6,4.4,0.383333,0.76,15.8,0.0,11.669166,21.40,11.0,20.0


In [12]:
la2 = la1[(la1['Year'] > 2009) & (la1['Year'] < 2018)]

In [13]:
la2 = la2.truncate(after='20170909') # limit the record before 20170909

In [14]:
crime_file = 'Crime_Data_2010_2017.csv'
j = pd.read_csv(crime_file,
               sep=',')
j

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,1208575,03/14/2013,03/11/2013,1800,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,"(33.9829, -118.3338)"
1,102005556,01/25/2010,01/22/2010,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"(34.0454, -118.3157)"
2,418,03/19/2013,03/18/2013,2030,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,"(33.942, -118.2717)"
3,101822289,11/11/2010,11/10/2010,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"(33.9572, -118.2717)"
4,42104479,01/11/2014,01/04/2014,2300,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,"(34.2009, -118.6369)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1584311,172116482,09/07/2017,09/07/2017,1005,21,Topanga,2141,440,THEFT PLAIN - PETTY ($950 & UNDER),0344,...,NaN,IC,Invest Cont,440.0,NaN,NaN,NaN,23800 CALVERT ST,NaN,"(34.1883, -118.6536)"
1584312,172116489,09/07/2017,09/07/2017,1200,21,Topanga,2156,330,BURGLARY FROM VEHICLE,0344 1605,...,NaN,IC,Invest Cont,330.0,NaN,NaN,NaN,6600 TOPANGA CANYON BL,NaN,"(34.1905, -118.6059)"
1584313,172116491,08/30/2017,07/15/2017,1,21,Topanga,2147,649,DOCUMENT FORGERY / STOLEN FELONY,0930 1822,...,NaN,IC,Invest Cont,649.0,NaN,NaN,NaN,20900 SHERMAN WY,NaN,"(34.201, -118.5885)"
1584314,172116498,09/08/2017,09/08/2017,1845,21,Topanga,2136,946,OTHER MISCELLANEOUS CRIME,1402,...,NaN,IC,Invest Cont,946.0,998.0,NaN,NaN,REMMET,WYANDOTTE,"(34.2029, -118.6003)"


In [15]:
j['Date Occurred'] = pd.to_datetime(j['Date Occurred'])

In [16]:
count = j.groupby(['Date Occurred']).count()
count

,DR Number,Date Reported,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
Date Occurred,,,,,,,,,,,,,,,,,,,,,
2010-01-01,2135,2135,2135,2135,2135,2135,2135,2132,1985,1863,...,582,2135,2135,2135,124,4,0,2135,146,2135
2010-01-02,533,533,533,533,533,533,533,533,463,426,...,193,533,533,533,50,0,0,533,101,533
2010-01-03,539,539,539,539,539,539,539,539,468,435,...,203,539,539,539,41,0,0,539,102,539
2010-01-04,558,558,558,558,558,558,558,558,493,461,...,171,558,558,558,40,0,0,558,89,558
2010-01-05,546,546,546,546,546,546,546,546,478,446,...,154,546,546,546,39,0,0,546,70,546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05,441,441,441,441,441,441,441,441,392,405,...,176,441,441,441,18,0,0,441,94,441
2017-09-06,361,361,361,361,361,361,361,361,312,334,...,128,361,361,361,24,2,1,361,61,361
2017-09-07,242,242,242,242,242,242,242,242,223,228,...,115,242,242,242,18,1,0,242,46,242


In [17]:
la2.dtypes

Year                 float64
Month                float64
Day                  float64
O3 Mean              float64
O3 1st Max Value     float64
O3 1st Max Hour      float64
O3 AQI               float64
CO Mean              float64
CO 1st Max Value     float64
CO 1st Max Hour      float64
CO AQI               float64
SO2 Mean             float64
SO2 1st Max Value    float64
SO2 1st Max Hour     float64
SO2 AQI              float64
NO2 Mean             float64
NO2 1st Max Value    float64
NO2 1st Max Hour     float64
NO2 AQI              float64
dtype: object

In [18]:
count

,DR Number,Date Reported,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
Date Occurred,,,,,,,,,,,,,,,,,,,,,
2010-01-01,2135,2135,2135,2135,2135,2135,2135,2132,1985,1863,...,582,2135,2135,2135,124,4,0,2135,146,2135
2010-01-02,533,533,533,533,533,533,533,533,463,426,...,193,533,533,533,50,0,0,533,101,533
2010-01-03,539,539,539,539,539,539,539,539,468,435,...,203,539,539,539,41,0,0,539,102,539
2010-01-04,558,558,558,558,558,558,558,558,493,461,...,171,558,558,558,40,0,0,558,89,558
2010-01-05,546,546,546,546,546,546,546,546,478,446,...,154,546,546,546,39,0,0,546,70,546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05,441,441,441,441,441,441,441,441,392,405,...,176,441,441,441,18,0,0,441,94,441
2017-09-06,361,361,361,361,361,361,361,361,312,334,...,128,361,361,361,24,2,1,361,61,361
2017-09-07,242,242,242,242,242,242,242,242,223,228,...,115,242,242,242,18,1,0,242,46,242


In [19]:
crime_count = count['DR Number']

In [20]:
crime_count.mean()

564.0142399430403

In [21]:
c_poll = pd.concat([la2,crime_count],axis=1) # to join two dataframe
c_poll

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
2010-01-01,2010.0,1.0,1.0,0.017176,0.030000,15.500000,27.500000,0.463889,0.750000,14.500000,8.500000,0.200000,1.000000,10.500000,1.000000,23.152174,40.100000,19.500000,38.000000,2135
2010-01-02,2010.0,1.0,2.0,0.015618,0.025000,9.000000,23.000000,0.502084,0.800000,5.500000,9.000000,0.169565,0.850000,8.500000,0.500000,24.689130,37.850000,11.500000,35.500000,533
2010-01-03,2010.0,1.0,3.0,0.016323,0.030000,9.000000,28.000000,0.427083,0.650000,11.500000,7.500000,0.156522,0.650000,11.000000,0.500000,23.013340,37.000000,19.500000,35.000000,539
2010-01-04,2010.0,1.0,4.0,0.010030,0.020000,9.500000,18.500000,0.535417,0.800000,11.500000,9.000000,0.484783,1.350000,19.000000,1.500000,29.378260,43.450000,19.500000,40.500000,558
2010-01-05,2010.0,1.0,5.0,0.008705,0.018000,10.000000,17.000000,0.597917,1.050000,23.000000,12.000000,0.604348,1.900000,22.000000,2.000000,33.497727,49.700000,20.500000,46.500000,546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05,2017.0,9.0,5.0,0.023922,0.040000,7.666667,37.000000,0.208333,0.266667,8.666667,3.000000,0.125000,0.333333,21.666667,0.000000,8.359343,25.500000,22.333333,23.666667,441
2017-09-06,2017.0,9.0,6.0,0.022132,0.038667,9.666667,35.666667,0.339815,0.500000,5.666667,5.666667,0.249603,0.866667,14.333333,0.333333,16.692460,34.766667,12.333333,32.666667,361
2017-09-07,2017.0,9.0,7.0,0.026882,0.041333,9.666667,38.333333,0.383333,0.600000,6.666667,6.666667,0.200000,0.700000,12.666667,0.333333,16.720833,41.466667,5.333333,38.666667,242
2017-09-08,2017.0,9.0,8.0,0.033157,0.039000,11.333333,36.000000,0.263889,0.333333,4.000000,4.000000,0.165278,0.766667,11.000000,0.000000,11.447222,22.333333,5.000000,20.333333,138


In [22]:
c_poll['DR Number']=c_poll[['DR Number']].apply(min_max_scaler) 
# data normlaization,to let the data values in one same scale


In [25]:
c_poll.corr(method='spearman') # the corr() function, and the result is not clear. 
#Correlation(~Case number): 
#SO2 Mean: 0.19
#O3 Mean: 0.11
#CO Mean: -0.13
#NO2 Mean: -0.06


,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,-0.069773,-0.003333,0.185852,0.226447,-0.005521,0.225828,-0.132626,-0.127620,-0.023269,-0.120206,0.095208,-0.083259,0.126792,-0.178278,-0.104969,-0.120211,-0.023333,-0.121078,0.363531
Month,-0.069773,1.000000,0.010271,-0.038860,-0.000434,-0.121973,-0.000662,-0.044341,-0.022375,0.027911,-0.023263,-0.042967,-0.038570,0.044667,-0.041891,-0.019874,-0.064107,0.009209,-0.064163,0.025318
Day,-0.003333,0.010271,1.000000,0.024585,0.018691,0.008093,0.019053,-0.024446,-0.017761,-0.007130,-0.018057,-0.002347,-0.008173,-0.007045,-0.011981,-0.029551,-0.027002,-0.035930,-0.026452,-0.015490
O3 Mean,0.185852,-0.038860,0.024585,1.000000,0.853453,0.104918,0.851621,-0.493271,-0.532231,-0.186369,-0.534149,-0.333225,-0.289671,-0.159282,-0.264259,-0.616912,-0.503432,-0.367312,-0.503167,0.116913
O3 1st Max Value,0.226447,-0.000434,0.018691,0.853453,1.000000,0.048675,0.999324,-0.299617,-0.317457,-0.091513,-0.319007,-0.164262,-0.139347,-0.059815,-0.129967,-0.346652,-0.213132,-0.163975,-0.213177,0.154393
O3 1st Max Hour,-0.005521,-0.121973,0.008093,0.104918,0.048675,1.000000,0.048960,0.088642,0.081879,0.022058,0.081520,0.022290,0.053434,-0.042480,0.057244,0.107592,0.102002,-0.089980,0.102430,0.022717
O3 AQI,0.225828,-0.000662,0.019053,0.851621,0.999324,0.048960,1.000000,-0.299301,-0.317073,-0.091592,-0.318740,-0.163040,-0.138103,-0.058232,-0.129195,-0.345471,-0.212242,-0.163324,-0.212276,0.155521
CO Mean,-0.132626,-0.044341,-0.024446,-0.493271,-0.299617,0.088642,-0.299301,1.000000,0.969360,0.099499,0.967611,0.568597,0.531803,0.147021,0.496181,0.835419,0.779820,0.201395,0.779476,-0.135411
CO 1st Max Value,-0.127620,-0.022375,-0.017761,-0.532231,-0.317457,0.081879,-0.317073,0.969360,1.000000,0.162639,0.998119,0.586153,0.557823,0.167743,0.522416,0.865529,0.836439,0.209139,0.836012,-0.133679
CO 1st Max Hour,-0.023269,0.027911,-0.007130,-0.186369,-0.091513,0.022058,-0.091592,0.099499,0.162639,1.000000,0.163775,0.103307,0.109034,0.145517,0.092863,0.190694,0.212209,0.226303,0.212032,0.029247


In [27]:
c_poll[c_poll['SO2 Mean' ] > 0.7].corr(method='spearman')
# adding some limitions, with a better result.

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,0.058354,-0.037957,0.143968,0.201705,-0.060114,0.199701,-0.058726,-0.063732,-0.064734,-0.065687,-0.239411,-0.314155,0.109876,-0.337711,-0.063207,-0.095478,-0.049407,-0.099057,0.158154
Month,0.058354,1.000000,-0.019393,0.054438,0.090911,-0.164395,0.091093,-0.089947,-0.106017,-0.058166,-0.109742,0.071049,-0.091958,0.105198,-0.121431,-0.036489,-0.082797,0.020139,-0.082516,0.081972
Day,-0.037957,-0.019393,1.000000,-0.071122,-0.074028,-0.037271,-0.072606,-0.033229,-0.011822,-0.005750,-0.011607,-0.054903,0.043809,0.150365,0.060244,0.003075,0.023628,0.016442,0.026324,-0.023031
O3 Mean,0.143968,0.054438,-0.071122,1.000000,0.952122,0.348996,0.951495,-0.715737,-0.718713,-0.133550,-0.714753,-0.202894,-0.009679,-0.069135,-0.028990,-0.739508,-0.407785,-0.328803,-0.402950,0.331169
O3 1st Max Value,0.201705,0.090911,-0.074028,0.952122,1.000000,0.304380,0.999509,-0.682054,-0.689935,-0.131561,-0.686717,-0.239603,-0.063205,-0.059557,-0.073493,-0.662418,-0.325124,-0.209044,-0.321472,0.359505
O3 1st Max Hour,-0.060114,-0.164395,-0.037271,0.348996,0.304380,1.000000,0.301692,-0.152772,-0.173272,-0.111546,-0.169110,-0.134623,0.112854,-0.146139,0.134953,-0.222883,-0.022950,-0.066372,-0.020199,0.103152
O3 AQI,0.199701,0.091093,-0.072606,0.951495,0.999509,0.301692,1.000000,-0.685267,-0.692671,-0.128153,-0.689552,-0.237693,-0.063274,-0.057439,-0.073598,-0.665191,-0.329656,-0.208781,-0.325911,0.359578
CO Mean,-0.058726,-0.089947,-0.033229,-0.715737,-0.682054,-0.152772,-0.685267,1.000000,0.965373,-0.017314,0.965756,0.177050,0.079712,-0.014965,0.093538,0.833263,0.637430,0.244751,0.632198,-0.359142
CO 1st Max Value,-0.063732,-0.106017,-0.011822,-0.718713,-0.689935,-0.173272,-0.692671,0.965373,1.000000,0.027740,0.998954,0.168394,0.095593,0.007079,0.115644,0.825956,0.658321,0.225401,0.652930,-0.359459
CO 1st Max Hour,-0.064734,-0.058166,-0.005750,-0.133550,-0.131561,-0.111546,-0.128153,-0.017314,0.027740,1.000000,0.029444,0.026997,0.011170,0.092043,-0.019606,0.050036,0.090921,0.105570,0.088658,0.016086


In [28]:
c_poll1=c_poll.rename(index = str, columns={"DR Number":"total_crime","O3 Mean" : "O3_Mean","CO Mean":"CO_Mean","SO2 Mean":"SO2_Mean","NO2 Mean":"NO2_Mean"})

# change the column's name, using for ols model.

In [29]:
c_poll1
test = c_poll1[(c_poll1['O3_Mean' ]>0.014) & (c_poll1['SO2_Mean' ] > 0.6)]
test
# for test

,Year,Month,Day,O3_Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO_Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2_Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2_Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,total_crime
2010-01-12 00:00:00,2010.0,1.0,12.0,0.017139,0.021500,20.000000,20.000000,0.768333,1.000000,0.000000,11.500000,1.116063,3.750000,7.000000,5.000000,29.289946,48.350000,5.500000,46.500000,0.265877
2010-01-17 00:00:00,2010.0,1.0,17.0,0.020824,0.029500,9.000000,27.500000,0.527084,0.850000,2.500000,9.500000,0.669566,2.200000,8.000000,2.500000,23.113043,37.600000,1.500000,35.500000,0.250711
2010-01-28 00:00:00,2010.0,1.0,28.0,0.015474,0.028000,10.500000,26.000000,0.652500,0.900000,4.000000,10.000000,1.017391,2.900000,8.500000,3.500000,29.645653,44.150000,14.500000,41.500000,0.266825
2010-02-02 00:00:00,2010.0,2.0,2.0,0.014794,0.027500,11.500000,25.500000,0.775000,1.050000,5.500000,12.000000,1.043478,2.500000,10.500000,3.500000,32.271838,47.200000,13.500000,44.500000,0.262085
2010-02-03 00:00:00,2010.0,2.0,3.0,0.017236,0.027500,10.500000,25.500000,0.702083,1.050000,4.500000,12.000000,0.739131,1.500000,4.500000,2.000000,28.517193,43.700000,6.500000,41.000000,0.254976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-29 00:00:00,2017.0,8.0,29.0,0.038839,0.070333,11.666667,112.666667,0.436966,0.566667,3.333333,6.333333,0.643055,1.300000,11.000000,0.666667,22.305555,33.533333,15.666667,31.333333,0.248341
2017-08-30 00:00:00,2017.0,8.0,30.0,0.040078,0.068333,11.000000,99.000000,0.379167,0.533333,6.666667,6.000000,0.688889,2.100000,11.000000,2.333333,22.606401,45.133333,6.666667,42.000000,0.242180
2017-08-31 00:00:00,2017.0,8.0,31.0,0.036353,0.067000,9.666667,101.333333,0.391667,0.566667,9.000000,6.333333,0.737500,1.333333,6.666667,1.666667,24.666667,47.966667,6.000000,45.000000,0.285308
2017-09-01 00:00:00,2017.0,9.0,1.0,0.039079,0.068333,10.333333,100.666667,0.319444,0.466667,8.666667,5.333333,0.710749,1.666667,15.666667,1.666667,23.003261,48.866667,22.000000,46.000000,0.280095


In [30]:
crime_model = ols('total_crime ~ CO_Mean+O3_Mean+SO2_Mean+NO2_Mean', data=c_poll1).fit()
# ols regression


In [31]:
crime_summary = crime_model.summary() # print the result. 
crime_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_crime   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     22.72
Date:                Fri, 06 May 2022   Prob (F-statistic):           1.76e-18
Time:                        01:06:01   Log-Likelihood:                 4697.9
No. Observations:                2665   AIC:                            -9386.
Df Residuals:                    2660   BIC:                            -9356.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2524      0.005     55.047      0.000       0.243       0.261
CO_Mean       -0.0358      0.006     -6.051      0.000      -0.047      -0.024
O3_Mean        0.1938      0.107      1.817      0.069      -0.015       0.403
SO2_Mean       0.0267      0.003      7.764      0.000       0.020       0.033
NO2_Mean       0.0002      0.000      1.409      0.159   -9.45e-05       0.001
==============================================================================
Omnibus:                     3260.099   Durbin-Watson:                   1.596
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           883499.714
Skew:                           6.192   Prob(JB):                         0.00
Kurtosis:                      91.335   Cond. No.                     2.69e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.69e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [33]:
crime_model1 = ols('total_crime ~ CO_Mean+O3_Mean+SO2_Mean+NO2_Mean', data=test).fit()
crime_summary1 = crime_model1.summary()
crime_summary1

# the test model, also the result is bad >_<

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_crime   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     3.720
Date:                Fri, 06 May 2022   Prob (F-statistic):            0.00561
Time:                        01:06:33   Log-Likelihood:                 536.26
No. Observations:                 340   AIC:                            -1063.
Df Residuals:                     335   BIC:                            -1043.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3011      0.018     16.445      0.000       0.265       0.337
CO_Mean       -0.0383      0.017     -2.270      0.024      -0.071      -0.005
O3_Mean       -0.0121      0.374     -0.032      0.974      -0.748       0.724
SO2_Mean      -0.0155      0.011     -1.442      0.150      -0.037       0.006
NO2_Mean    5.704e-06      0.000      0.012      0.991      -0.001       0.001
==============================================================================
Omnibus:                      530.718   Durbin-Watson:                   1.910
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           154635.124
Skew:                           8.085   Prob(JB):                         0.00
Kurtosis:                     106.218   Cond. No.                     3.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [68]:
j.groupby(['Weapon Description']).count().sort_values('DR Number',ascending=False)
# showing the case number with different description which is larger

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Used Code,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
Weapon Description,,,,,,,,,,,,,,,,,,,,,
"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",319818,319818,319818,319818,319818,319818,319818,319818,319716,316845,...,319818,319818,319818,319817,19564,702,27,319818,78061,319816
VERBAL THREAT,43814,43814,43814,43814,43814,43814,43814,43814,43783,42710,...,43814,43814,43814,43813,2627,113,4,43814,5750,43814
UNKNOWN WEAPON/OTHER WEAPON,40746,40746,40746,40746,40746,40746,40746,40746,40739,40190,...,40746,40746,40746,40746,4133,67,2,40746,8275,40746
HAND GUN,25352,25352,25352,25352,25352,25352,25352,25352,25345,24999,...,25352,25352,25352,25351,5848,176,5,25352,10228,25352
SEMI-AUTOMATIC PISTOL,10096,10096,10096,10096,10096,10096,10096,10096,10096,10004,...,10096,10096,10096,10096,2377,59,3,10096,3936,10096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAC-10 SEMIAUTOMATIC ASSAULT WEAPON,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,2,0,0,8,5,8
ANTIQUE FIREARM,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,0,0,0,7,2,7
MAC-11 SEMIAUTOMATIC ASSAULT WEAPON,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,1,0,0,5,2,5


In [72]:
fist = j[j['Weapon Description'] == 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)']
f_count = fist.groupby(['Date Occurred']).count()
f_crime_count = f_count['DR Number']
f_c_poll = pd.concat([la2,f_crime_count],axis=1)
f_c_poll['DR Number']=f_c_poll[['DR Number']].apply(min_max_scaler)

# set limitation on crime dataset, the previous is execuate on air quality set.

In [73]:
f_count

,DR Number,Date Reported,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
Date Occurred,,,,,,,,,,,,,,,,,,,,,
2010-01-01,453,453,453,453,453,453,453,450,438,346,...,453,453,453,453,76,2,0,453,62,453
2010-01-02,123,123,123,123,123,123,123,123,120,118,...,123,123,123,123,8,0,0,123,40,123
2010-01-03,123,123,123,123,123,123,123,123,120,120,...,123,123,123,123,6,0,0,123,32,123
2010-01-04,100,100,100,100,100,100,100,100,99,95,...,100,100,100,100,7,0,0,100,29,100
2010-01-05,87,87,87,87,87,87,87,87,87,83,...,87,87,87,87,5,0,0,87,16,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05,99,99,99,99,99,99,99,99,99,96,...,99,99,99,99,6,0,0,99,25,99
2017-09-06,85,85,85,85,85,85,85,85,85,83,...,85,85,85,85,7,1,1,85,11,85
2017-09-07,68,68,68,68,68,68,68,68,67,68,...,68,68,68,68,1,0,0,68,14,68


In [75]:
f_c_poll.corr(method='spearman') # which the result is better than before. 

,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,-0.069773,-0.003333,0.185852,0.226447,-0.005521,0.225828,-0.132626,-0.127620,-0.023269,-0.120206,0.095208,-0.083259,0.126792,-0.178278,-0.104969,-0.120211,-0.023333,-0.121078,0.124960
Month,-0.069773,1.000000,0.010271,-0.038860,-0.000434,-0.121973,-0.000662,-0.044341,-0.022375,0.027911,-0.023263,-0.042967,-0.038570,0.044667,-0.041891,-0.019874,-0.064107,0.009209,-0.064163,0.048140
Day,-0.003333,0.010271,1.000000,0.024585,0.018691,0.008093,0.019053,-0.024446,-0.017761,-0.007130,-0.018057,-0.002347,-0.008173,-0.007045,-0.011981,-0.029551,-0.027002,-0.035930,-0.026452,-0.025284
O3 Mean,0.185852,-0.038860,0.024585,1.000000,0.853453,0.104918,0.851621,-0.493271,-0.532231,-0.186369,-0.534149,-0.333225,-0.289671,-0.159282,-0.264259,-0.616912,-0.503432,-0.367312,-0.503167,0.302996
O3 1st Max Value,0.226447,-0.000434,0.018691,0.853453,1.000000,0.048675,0.999324,-0.299617,-0.317457,-0.091513,-0.319007,-0.164262,-0.139347,-0.059815,-0.129967,-0.346652,-0.213132,-0.163975,-0.213177,0.366476
O3 1st Max Hour,-0.005521,-0.121973,0.008093,0.104918,0.048675,1.000000,0.048960,0.088642,0.081879,0.022058,0.081520,0.022290,0.053434,-0.042480,0.057244,0.107592,0.102002,-0.089980,0.102430,-0.068532
O3 AQI,0.225828,-0.000662,0.019053,0.851621,0.999324,0.048960,1.000000,-0.299301,-0.317073,-0.091592,-0.318740,-0.163040,-0.138103,-0.058232,-0.129195,-0.345471,-0.212242,-0.163324,-0.212276,0.367333
CO Mean,-0.132626,-0.044341,-0.024446,-0.493271,-0.299617,0.088642,-0.299301,1.000000,0.969360,0.099499,0.967611,0.568597,0.531803,0.147021,0.496181,0.835419,0.779820,0.201395,0.779476,-0.167065
CO 1st Max Value,-0.127620,-0.022375,-0.017761,-0.532231,-0.317457,0.081879,-0.317073,0.969360,1.000000,0.162639,0.998119,0.586153,0.557823,0.167743,0.522416,0.865529,0.836439,0.209139,0.836012,-0.177018
CO 1st Max Hour,-0.023269,0.027911,-0.007130,-0.186369,-0.091513,0.022058,-0.091592,0.099499,0.162639,1.000000,0.163775,0.103307,0.109034,0.145517,0.092863,0.190694,0.212209,0.226303,0.212032,-0.155023


In [85]:
f_c_poll[(f_c_poll['O3 Mean' ]>0.0015) & (f_c_poll['SO2 Mean' ] > 0.7)].corr(method='spearman')

# Where is the best result. 
#Correlation(~Case number): Setting SO2 > 0.7 & ‘Crime Description’ like ‘Weapon’ 
#SO2 Mean: -0.13
#O3 Mean: 0.51 vs 0.11                           4 times than before(all cases)
#CO Mean: -0.45 vs -0.13                        3 times than before
#NO2 Mean: -0.43 vs -0.06                      7 times than before


,Year,Month,Day,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,DR Number
Year,1.000000,0.058354,-0.037957,0.143968,0.201705,-0.060114,0.199701,-0.058726,-0.063732,-0.064734,-0.065687,-0.239411,-0.314155,0.109876,-0.337711,-0.063207,-0.095478,-0.049407,-0.099057,0.053852
Month,0.058354,1.000000,-0.019393,0.054438,0.090911,-0.164395,0.091093,-0.089947,-0.106017,-0.058166,-0.109742,0.071049,-0.091958,0.105198,-0.121431,-0.036489,-0.082797,0.020139,-0.082516,0.093034
Day,-0.037957,-0.019393,1.000000,-0.071122,-0.074028,-0.037271,-0.072606,-0.033229,-0.011822,-0.005750,-0.011607,-0.054903,0.043809,0.150365,0.060244,0.003075,0.023628,0.016442,0.026324,-0.001590
O3 Mean,0.143968,0.054438,-0.071122,1.000000,0.952122,0.348996,0.951495,-0.715737,-0.718713,-0.133550,-0.714753,-0.202894,-0.009679,-0.069135,-0.028990,-0.739508,-0.407785,-0.328803,-0.402950,0.516001
O3 1st Max Value,0.201705,0.090911,-0.074028,0.952122,1.000000,0.304380,0.999509,-0.682054,-0.689935,-0.131561,-0.686717,-0.239603,-0.063205,-0.059557,-0.073493,-0.662418,-0.325124,-0.209044,-0.321472,0.547984
O3 1st Max Hour,-0.060114,-0.164395,-0.037271,0.348996,0.304380,1.000000,0.301692,-0.152772,-0.173272,-0.111546,-0.169110,-0.134623,0.112854,-0.146139,0.134953,-0.222883,-0.022950,-0.066372,-0.020199,0.136901
O3 AQI,0.199701,0.091093,-0.072606,0.951495,0.999509,0.301692,1.000000,-0.685267,-0.692671,-0.128153,-0.689552,-0.237693,-0.063274,-0.057439,-0.073598,-0.665191,-0.329656,-0.208781,-0.325911,0.548733
CO Mean,-0.058726,-0.089947,-0.033229,-0.715737,-0.682054,-0.152772,-0.685267,1.000000,0.965373,-0.017314,0.965756,0.177050,0.079712,-0.014965,0.093538,0.833263,0.637430,0.244751,0.632198,-0.454969
CO 1st Max Value,-0.063732,-0.106017,-0.011822,-0.718713,-0.689935,-0.173272,-0.692671,0.965373,1.000000,0.027740,0.998954,0.168394,0.095593,0.007079,0.115644,0.825956,0.658321,0.225401,0.652930,-0.460221
CO 1st Max Hour,-0.064734,-0.058166,-0.005750,-0.133550,-0.131561,-0.111546,-0.128153,-0.017314,0.027740,1.000000,0.029444,0.026997,0.011170,0.092043,-0.019606,0.050036,0.090921,0.105570,0.088658,-0.096447


In [77]:
f_c_poll1=f_c_poll.rename(index = str, columns={"DR Number":"total_crime","O3 Mean" : "O3_Mean","CO Mean":"CO_Mean","SO2 Mean":"SO2_Mean","NO2 Mean":"NO2_Mean"})

In [78]:
c_poll1

,Year,Month,Day,O3_Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO_Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2_Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2_Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,total_crime
2010-01-01 00:00:00,2010.0,1.0,1.0,0.017176,0.030000,15.500000,27.500000,0.463889,0.750000,14.500000,8.500000,0.200000,1.000000,10.500000,1.000000,23.152174,40.100000,19.500000,38.000000,1.000000
2010-01-02 00:00:00,2010.0,1.0,2.0,0.015618,0.025000,9.000000,23.000000,0.502084,0.800000,5.500000,9.000000,0.169565,0.850000,8.500000,0.500000,24.689130,37.850000,11.500000,35.500000,0.240758
2010-01-03 00:00:00,2010.0,1.0,3.0,0.016323,0.030000,9.000000,28.000000,0.427083,0.650000,11.500000,7.500000,0.156522,0.650000,11.000000,0.500000,23.013340,37.000000,19.500000,35.000000,0.243602
2010-01-04 00:00:00,2010.0,1.0,4.0,0.010030,0.020000,9.500000,18.500000,0.535417,0.800000,11.500000,9.000000,0.484783,1.350000,19.000000,1.500000,29.378260,43.450000,19.500000,40.500000,0.252607
2010-01-05 00:00:00,2010.0,1.0,5.0,0.008705,0.018000,10.000000,17.000000,0.597917,1.050000,23.000000,12.000000,0.604348,1.900000,22.000000,2.000000,33.497727,49.700000,20.500000,46.500000,0.246919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05 00:00:00,2017.0,9.0,5.0,0.023922,0.040000,7.666667,37.000000,0.208333,0.266667,8.666667,3.000000,0.125000,0.333333,21.666667,0.000000,8.359343,25.500000,22.333333,23.666667,0.197156
2017-09-06 00:00:00,2017.0,9.0,6.0,0.022132,0.038667,9.666667,35.666667,0.339815,0.500000,5.666667,5.666667,0.249603,0.866667,14.333333,0.333333,16.692460,34.766667,12.333333,32.666667,0.159242
2017-09-07 00:00:00,2017.0,9.0,7.0,0.026882,0.041333,9.666667,38.333333,0.383333,0.600000,6.666667,6.666667,0.200000,0.700000,12.666667,0.333333,16.720833,41.466667,5.333333,38.666667,0.102844
2017-09-08 00:00:00,2017.0,9.0,8.0,0.033157,0.039000,11.333333,36.000000,0.263889,0.333333,4.000000,4.000000,0.165278,0.766667,11.000000,0.000000,11.447222,22.333333,5.000000,20.333333,0.053555


In [79]:
f_c_poll1

,Year,Month,Day,O3_Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,CO_Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,SO2_Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,NO2_Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,total_crime
2010-01-01 00:00:00,2010.0,1.0,1.0,0.017176,0.030000,15.500000,27.500000,0.463889,0.750000,14.500000,8.500000,0.200000,1.000000,10.500000,1.000000,23.152174,40.100000,19.500000,38.000000,1.000000
2010-01-02 00:00:00,2010.0,1.0,2.0,0.015618,0.025000,9.000000,23.000000,0.502084,0.800000,5.500000,9.000000,0.169565,0.850000,8.500000,0.500000,24.689130,37.850000,11.500000,35.500000,0.251701
2010-01-03 00:00:00,2010.0,1.0,3.0,0.016323,0.030000,9.000000,28.000000,0.427083,0.650000,11.500000,7.500000,0.156522,0.650000,11.000000,0.500000,23.013340,37.000000,19.500000,35.000000,0.251701
2010-01-04 00:00:00,2010.0,1.0,4.0,0.010030,0.020000,9.500000,18.500000,0.535417,0.800000,11.500000,9.000000,0.484783,1.350000,19.000000,1.500000,29.378260,43.450000,19.500000,40.500000,0.199546
2010-01-05 00:00:00,2010.0,1.0,5.0,0.008705,0.018000,10.000000,17.000000,0.597917,1.050000,23.000000,12.000000,0.604348,1.900000,22.000000,2.000000,33.497727,49.700000,20.500000,46.500000,0.170068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-05 00:00:00,2017.0,9.0,5.0,0.023922,0.040000,7.666667,37.000000,0.208333,0.266667,8.666667,3.000000,0.125000,0.333333,21.666667,0.000000,8.359343,25.500000,22.333333,23.666667,0.197279
2017-09-06 00:00:00,2017.0,9.0,6.0,0.022132,0.038667,9.666667,35.666667,0.339815,0.500000,5.666667,5.666667,0.249603,0.866667,14.333333,0.333333,16.692460,34.766667,12.333333,32.666667,0.165533
2017-09-07 00:00:00,2017.0,9.0,7.0,0.026882,0.041333,9.666667,38.333333,0.383333,0.600000,6.666667,6.666667,0.200000,0.700000,12.666667,0.333333,16.720833,41.466667,5.333333,38.666667,0.126984
2017-09-08 00:00:00,2017.0,9.0,8.0,0.033157,0.039000,11.333333,36.000000,0.263889,0.333333,4.000000,4.000000,0.165278,0.766667,11.000000,0.000000,11.447222,22.333333,5.000000,20.333333,0.043084


In [80]:
f_crime_model = ols('total_crime ~ CO_Mean+O3_Mean+SO2_Mean+NO2_Mean', data=f_c_poll1).fit()
f_crime_summary = f_crime_model.summary()
f_crime_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_crime   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     50.19
Date:                Mon, 02 May 2022   Prob (F-statistic):           8.81e-41
Time:                        20:33:35   Log-Likelihood:                 4174.9
No. Observations:                2665   AIC:                            -8340.
Df Residuals:                    2660   BIC:                            -8310.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2045      0.006     36.656      0.000       0.194       0.215
CO_Mean       -0.0125      0.007     -1.734      0.083      -0.027       0.002
O3_Mean        1.1012      0.130      8.484      0.000       0.847       1.356
SO2_Mean       0.0280      0.004      6.688      0.000       0.020       0.036
NO2_Mean      -0.0005      0.000     -2.202      0.028      -0.001   -5.03e-05
==============================================================================
Omnibus:                     2278.937   Durbin-Watson:                   1.513
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           180277.531
Skew:                           3.594   Prob(JB):                         0.00
Kurtosis:                      42.647   Cond. No.                     2.69e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.69e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [81]:
test1 = f_c_poll1[(f_c_poll1['O3_Mean' ]>0.016) & (f_c_poll1['SO2_Mean' ] > 0.7)]
f_crime_model1 = ols('total_crime ~ CO_Mean+O3_Mean+SO2_Mean+NO2_Mean', data=test1).fit()
f_crime_summary1 = f_crime_model1.summary()
f_crime_summary1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            total_crime   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     6.231
Date:                Mon, 02 May 2022   Prob (F-statistic):           9.77e-05
Time:                        21:49:22   Log-Likelihood:                 346.18
No. Observations:                 198   AIC:                            -682.4
Df Residuals:                     193   BIC:                            -665.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2676      0.021     12.635      0.000       0.226       0.309
CO_Mean       -0.0438      0.016     -2.672      0.008      -0.076      -0.011
O3_Mean        0.5894      0.420      1.404      0.162      -0.239       1.417
SO2_Mean      -0.0121      0.011     -1.137      0.257      -0.033       0.009
NO2_Mean   -8.513e-05      0.000     -0.170      0.865      -0.001       0.001
==============================================================================
Omnibus:                       43.251   Durbin-Watson:                   1.913
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               82.274
Skew:                           1.067   Prob(JB):                     1.36e-18
Kurtosis:                       5.327   Cond. No.                     3.59e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.59e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
# above two is the ols model with two some case data, and the result is improved but not good enough, so cannot
# support there is a relation through ols.